In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0.0,0,0.0,0
1,2020-01-01,Afghanistan,0.0,0,0.0,0
2,2020-01-02,Afghanistan,0.0,0,0.0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-27,France,125706.0,0.0,23226.0,0.0,0.0,0.0
0,2020-04-27,China,84407.0,0.0,4673.0,0.0,0.0,0.0
0,2020-04-27,Italy,199956.0,0.0,27031.0,0.0,0.0,0.0
0,2020-04-27,Spain,208321.0,0.0,23372.0,0.0,0.0,0.0
0,2020-04-27,United States,1016390.0,0.0,56952.0,0.0,0.0,0.0
0,2020-04-27,World,3021107.0,0.0,211180.0,0.0,0.0,0.0
0,2020-04-27,United Kingdom,157587.0,0.0,21378.0,0.0,0.0,0.0
0,2020-04-27,Germany,156428.0,0.0,5863.0,0.0,0.0,0.0
0,2020-04-27,Iran,91607.0,0.0,5780.0,0.0,0.0,0.0
0,2020-04-27,Turkey,112255.0,0.0,2902.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-28,France,125706.0,128339.0,23226.0,23293.0,-2633.0,-67.0
0,2020-04-28,China,84407.0,83938.0,4673.0,4637.0,469.0,36.0
0,2020-04-28,Italy,199956.0,199414.0,27031.0,26977.0,542.0,54.0
0,2020-04-27,Spain,208321.0,210895.0,23372.0,23190.0,-2574.0,182.0
0,2020-04-28,United States,1016390.0,988451.0,56952.0,56245.0,27939.0,707.0
0,2020-04-28,World,3021107.0,2982688.0,211180.0,210193.0,38419.0,987.0
0,2020-04-28,United Kingdom,157587.0,157149.0,21378.0,21092.0,438.0,286.0
0,2020-04-28,Germany,156428.0,156337.0,5863.0,5913.0,91.0,-50.0
0,2020-04-28,Iran,91607.0,91472.0,5780.0,5806.0,135.0,-26.0
0,2020-04-28,Turkey,112255.0,112261.0,2902.0,2900.0,-6.0,2.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")